In [2]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
from tqdm import tqdm

c:\Users\Ankit\Desktop\SHL-GenAI-Assessment-Recommendation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
excel_path = os.path.join(parent_dir, "data", "Gen_AI_Dataset.xlsx")

test_df=pd.read_excel(excel_path,sheet_name="Test-Set")

In [4]:
test_df

,Query
0,Looking to hire mid-level professionals who ar...
1,Job Description\n\n Join a community that is s...
2,I am hiring for an analyst and wants applicati...
3,I have a JD Job Description\n\n People Science...
4,I am new looking for new graduates in my sales...
5,For Marketing - Content Writer Position\n\nDep...
6,I want to hire a product manager with 3-4 year...
7,Suggest me an assessment for the JD below Job ...
8,I want to hire Customer support executives who...


In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

os.chdir("..")
persist_dir = "app/chroma_shl_db"
vectordb = Chroma(
    persist_directory=persist_dir, 
    embedding_function=embedding_model
)


C:\Users\Ankit\AppData\Local\Temp\ipykernel_12492\4200642978.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


In [6]:
retriever = vectordb.as_retriever(
    search_type="mmr",          
    search_kwargs={
        "k": 10,  
        "fetch_k": 25, 
        "lambda_mult": 0.5,
        "score_threshold": 0.3  
    }
)

In [7]:
results = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    query = row["Query"]
    docs = retriever.invoke(query)
    
    for doc in docs:
        for line in doc.page_content.splitlines():
            if line.lower().startswith("url:"):
                url = line.split(":", 1)[1].strip()
                results.append({"Query": query, "Assessment_url": url})
                break

100%|██████████| 9/9 [00:00<00:00, 10.64it/s]


In [8]:
import os
import pandas as pd

os.makedirs("data", exist_ok=True)
output_path = os.path.join("data", "test_submission.csv")

submission_df = pd.DataFrame(results)
submission_df.to_csv(output_path, index=False, encoding="utf-8-sig")